In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import math

In [ ]:
def graph_json(filename):
    with open(filename) as json_file:
        json_data = json.load(json_file)
        name = json_data['ObjectName']
        seed = json_data['Seed']
        deformed_deviation = json_data['DeformedAngularDeviation']
        deformation_method = json_data['DeformationMethod']
        offsets = np.array(json_data['Offsets'], dtype='float')
        vertices = np.array(json_data['Vertices'], dtype='int')
        skipped_iterations = np.array(json_data['SkippedIterations'], dtype='bool')
        ideal_normal_angles_from_ray = np.array(json_data['IdealNormalAnglesFromRay'], dtype='float')
        iterations = vertices.size
        deviations = np.array(json_data['Deviations'], dtype='float').reshape((iterations, offsets.size))

        min_deviation = np.min(deviations[-1])
        max_deviation = np.max(deviations)
        min_offset = np.round(np.min(offsets), 2)
        max_offset = np.round(np.max(offsets), 2)
        effective_iterations = iterations - np.sum(skipped_iterations)

        deviation_diff_percent = (min_deviation-deformed_deviation)/deformed_deviation * 100

        info = f"{name} total angular deviation (0; {np.round(max_deviation,2)}) per offset ({min_offset}; {max_offset})"
        info += f"\n {iterations} iterations plotted ltr-ttb. {effective_iterations} iterations ({np.round(effective_iterations/iterations*100, 2)}%) reduced the minimum"
        info += f"\n Total angular deviation reduced from {np.round(deformed_deviation, 2)} to {np.round(min_deviation, 2)} ({np.round(deviation_diff_percent, 2)}%)"
        info += f"\n Deformation method: {deformation_method}"
        if (deformation_method == "Random"):
            info += f" (seed {seed})"
        info += "Legend"
        info += "-------------"
        info += "ndfr: ideal normal's degrees from ray -- used to see orthogonality to viewpoint"
        print(info)

        xplots = 10
        yplots = math.ceil(iterations/xplots)
        fig, axs = plt.subplots(yplots, xplots, figsize=(12, yplots*1.5))

        for y in range(yplots):
            for x in range(xplots):
                iter = y*xplots + x
                if iter < iterations:
                    color = 'red' if skipped_iterations[iter] else 'blue'
                    axs[y,x].plot(offsets, deviations[iter], c=color)
                    vertex = vertices[iter]
                    title = f"v: {vertex}"
                    title += f"\nndfr: {np.round(ideal_normal_angles_from_ray[vertex],1)}"
                    axs[y,x].set_title(title)

                axs[y,x].set_ylim(0, max_deviation)
                axs[y,x].set_xticklabels([])
                axs[y,x].set_yticklabels([])
        
        fig.tight_layout()
        fig.show()

In [ ]:
graph_json("Tetrahedron.json")

In [ ]:
graph_json("Cube.json")

In [ ]:
graph_json("Icosahedron.json")